In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.appName('Basics').getOrCreate()

df = spark.read.csv('healthcare-dataset-stroke-data.csv', inferSchema=True,header=True)

In [5]:
#remove columns with missing values

find_na = df.select([F.count(F.when(F.isnan(i) | \
                                   F.col(i).contains('N/A') | \
                                   F.col(i).contains('Unknown') | \
                                   F.col(i).isNull(), i)).alias(i) \
                    for i in df.columns])
find_na.show()


dropped = df.select([col for col in  df.columns if col not in ['bmi','smoking_status']])



dropped.show()

+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
| id|gender|age|hypertension|heart_disease|ever_married|work_type|Residence_type|avg_glucose_level|bmi|smoking_status|stroke|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+
|  0|     0|  0|           0|            0|           0|        0|             0|                0|201|          1544|     0|
+---+------+---+------------+-------------+------------+---------+--------------+-----------------+---+--------------+------+

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+------+
| 9046|  Male|67.0|        

In [6]:
#complete missing values with avg of the column 

#bmi
bmi_avg = str(round(df.agg({'bmi': 'avg'}).collect()[0][0],2))
df = df.drop('smoking_status').na.replace(['N/A'],[bmi_avg])
df.show()

+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+------+
|   id|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi|stroke|
+-----+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+------+
| 9046|  Male|67.0|           0|            1|         Yes|      Private|         Urban|           228.69| 36.6|     1|
|51676|Female|61.0|           0|            0|         Yes|Self-employed|         Rural|           202.21|28.89|     1|
|31112|  Male|80.0|           0|            1|         Yes|      Private|         Rural|           105.92| 32.5|     1|
|60182|Female|49.0|           0|            0|         Yes|      Private|         Urban|           171.23| 34.4|     1|
| 1665|Female|79.0|           1|            0|         Yes|Self-employed|         Rural|           174.12|   24|     1|
|56669|  Male|81.0|           0|        